# Data Exploration: Elliptic Dataset
# Augmentate network features
https://www.kaggle.com/datasets/ellipticco/elliptic-data-set/data

[1] Elliptic, www.elliptic.co.

[2] M. Weber, G. Domeniconi, J. Chen, D. K. I. Weidele, C. Bellei, T. Robinson, C. E. Leiserson, "Anti-Money Laundering in Bitcoin: Experimenting with Graph Convolutional Networks for Financial Forensics", KDD ’19 Workshop on Anomaly Detection in Finance, August 2019, Anchorage, AK, USA.

Description: The Elliptic Data Set maps Bitcoin transactions to real entities belonging to licit categories (exchanges, wallet providers, miners, licit services, etc.) versus illicit ones (scams, malware, terrorist organizations, ransomware, Ponzi schemes, etc.). The task on the dataset is to classify the illicit and licit nodes in the graph.


In [1]:
import pandas as pd
from src.features.network_features import *
from src.data.load_data import load_elliptic_dataset
pd.options.plotting.backend = 'plotly'
import random
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import torch



In [2]:
DATA_PATH = 'data/raw/'
nodes_df, edges_df = load_elliptic_dataset(DATA_PATH)
nodes_df['class_label'] = nodes_df['class_label'].replace(['1', '2', 'unknown'], ['illicit', 'licit', 'unknown'])

In [3]:
# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [4]:
G = build_nx_graph(edges_df)
print(f"Graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
labels = create_label_dict(nodes_df)
print("Building time step graphs...")
time_step_graphs = build_time_step_graphs(edges_df, nodes_df)
print(f"Created {len(time_step_graphs)} separate graphs for different time steps")

Graph has 203769 nodes and 234355 edges
Building time step graphs...
Created 49 separate graphs for different time steps


In [5]:
# 4. Compute GuiltyWalker features
print("Computing GuiltyWalker features...")
gw_features = guilty_walker_features(nodes_df, time_step_graphs)

Computing GuiltyWalker features...


Processing time steps: 100%|█████████████████████████████████████████████████| 49/49 [06:44<00:00,  8.25s/it]


In [6]:
gw_features.merge(nodes_df[['txId', 'class_label']], on = 'txId').query('class_label == "illicit" & gw_avg_step != 1')

,txId,gw_hit_ratio,gw_min_step,gw_avg_step,gw_unique_illicit,class_label
907,232629023,0.0,51,51.0,0,illicit
1361,230389796,0.0,51,51.0,0,illicit
2718,17387772,0.0,51,51.0,0,illicit
2815,232947878,0.0,51,51.0,0,illicit
3423,16754007,0.0,51,51.0,0,illicit
...,...,...,...,...,...,...
203685,159043651,0.0,51,51.0,0,illicit
203708,158360779,0.0,51,51.0,0,illicit
203736,159028476,0.0,51,51.0,0,illicit
203759,158375075,0.0,51,51.0,0,illicit


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
# Assuming nodes_df is your original dataframe
feature_cols = nodes_df.drop(columns=['txId', 'time_step', 'class_label']).columns.to_list()

# Add a SET column initialized with 'unknown' for all rows
nodes_df['SET'] = 'unknown'

# Split the labeled data indices
labeled_indices = nodes_df[nodes_df['class_label'] != 'unknown'].index
unlabeled_indices = nodes_df[nodes_df['class_label'] == 'unknown'].index

# Assign 'unlabeled' to all unlabeled observations
nodes_df.loc[unlabeled_indices, 'SET'] = 'train'

# Split the labeled indices into train, validation, and test sets
temp_indices, test_indices = train_test_split(
    labeled_indices, 
    test_size=0.2, 
    random_state=42, 
    stratify=nodes_df.loc[labeled_indices, 'class_label']
)

train_indices, val_indices = train_test_split(
    temp_indices, 
    test_size=0.25, 
    random_state=42, 
    stratify=nodes_df.loc[temp_indices, 'class_label']
)

# Assign the SET values
nodes_df.loc[train_indices, 'SET'] = 'train'
nodes_df.loc[val_indices, 'SET'] = 'validation'
nodes_df.loc[test_indices, 'SET'] = 'test'


In [9]:
train_df = nodes_df.copy() 
train_df.loc[train_df['SET'].isin(['validation', 'test']), 'class_label'] = 'unknown'


mps_device = torch.device("mps")
# 4. Train GCN models - they'll only learn from labeled nodes in train set
feature_cols = [col for col in nodes_df.columns if col not in ['txId', 'time_step', 'class_label', 'SET']]
train_embeddings, trained_models = train_gcn_get_embeddings(
    train_df, 
    time_step_graphs,
    feature_cols,
    device=mps_device, 
    epochs=400
)



GCN Embeddings:   0%|                                                                 | 0/49 [00:00<?, ?it/s]/Users/nick/AML/src/features/network_features.py:196: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_mask = torch.tensor(train_mask, dtype=torch.bool)


Time step 1, Epoch 0: Loss 11.7355
Time step 1, Epoch 10: Loss 4.1549
Time step 1, Epoch 20: Loss 0.6779
Time step 1, Epoch 30: Loss 0.2726
Time step 1, Epoch 40: Loss 0.1482
Time step 1, Epoch 50: Loss 0.1019
Time step 1, Epoch 60: Loss 0.0796
Time step 1, Epoch 70: Loss 0.0631
Time step 1, Epoch 80: Loss 0.0545
Time step 1, Epoch 90: Loss 0.0521
Time step 1, Epoch 100: Loss 0.0434
Time step 1, Epoch 110: Loss 0.0412
Time step 1, Epoch 120: Loss 0.0346
Time step 1, Epoch 130: Loss 0.0277
Time step 1, Epoch 140: Loss 0.0311
Time step 1, Epoch 150: Loss 0.0286
Time step 1, Epoch 160: Loss 0.0275
Time step 1, Epoch 170: Loss 0.0245
Time step 1, Epoch 180: Loss 0.0244
Time step 1, Epoch 190: Loss 0.0218
Time step 1, Epoch 200: Loss 0.0232
Time step 1, Epoch 210: Loss 0.0185
Time step 1, Epoch 220: Loss 0.0207
Time step 1, Epoch 230: Loss 0.0201
Time step 1, Epoch 240: Loss 0.0201
Time step 1, Epoch 250: Loss 0.0174
Time step 1, Epoch 260: Loss 0.0168
Time step 1, Epoch 270: Loss 0.0174
Ti

GCN Embeddings:   2%|█▏                                                       | 1/49 [00:13<10:27, 13.07s/it]/Users/nick/AML/src/features/network_features.py:196: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_mask = torch.tensor(train_mask, dtype=torch.bool)


Time step 2, Epoch 0: Loss 4.7589
Time step 2, Epoch 10: Loss 1.2878
Time step 2, Epoch 20: Loss 0.4482
Time step 2, Epoch 30: Loss 0.2141
Time step 2, Epoch 40: Loss 0.1373
Time step 2, Epoch 50: Loss 0.1035
Time step 2, Epoch 60: Loss 0.0761
Time step 2, Epoch 70: Loss 0.0631
Time step 2, Epoch 80: Loss 0.0572
Time step 2, Epoch 90: Loss 0.0467
Time step 2, Epoch 100: Loss 0.0476
Time step 2, Epoch 110: Loss 0.0399
Time step 2, Epoch 120: Loss 0.0356
Time step 2, Epoch 130: Loss 0.0339
Time step 2, Epoch 140: Loss 0.0301
Time step 2, Epoch 150: Loss 0.0328
Time step 2, Epoch 160: Loss 0.0265
Time step 2, Epoch 170: Loss 0.0279
Time step 2, Epoch 180: Loss 0.0268
Time step 2, Epoch 190: Loss 0.0239
Time step 2, Epoch 200: Loss 0.0217
Time step 2, Epoch 210: Loss 0.0188
Time step 2, Epoch 220: Loss 0.0175
Time step 2, Epoch 230: Loss 0.0205
Time step 2, Epoch 240: Loss 0.0140
Time step 2, Epoch 250: Loss 0.0147
Time step 2, Epoch 260: Loss 0.0134
Time step 2, Epoch 270: Loss 0.0135
Tim

GCN Embeddings:   4%|██▎                                                      | 2/49 [00:22<08:35, 10.96s/it]

...

Time step 48, Epoch 390: Loss 0.0099


/Users/nick/AML/src/features/network_features.py:196: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_mask = torch.tensor(train_mask, dtype=torch.bool)


Time step 49, Epoch 0: Loss 5.1402
Time step 49, Epoch 10: Loss 1.6031
Time step 49, Epoch 20: Loss 0.5574
Time step 49, Epoch 30: Loss 0.2959
Time step 49, Epoch 40: Loss 0.1839
Time step 49, Epoch 50: Loss 0.1354
Time step 49, Epoch 60: Loss 0.1190
Time step 49, Epoch 70: Loss 0.0952
Time step 49, Epoch 80: Loss 0.0719
Time step 49, Epoch 90: Loss 0.0628
Time step 49, Epoch 100: Loss 0.0575
Time step 49, Epoch 110: Loss 0.0483
Time step 49, Epoch 120: Loss 0.0577
Time step 49, Epoch 130: Loss 0.0400
Time step 49, Epoch 140: Loss 0.0382
Time step 49, Epoch 150: Loss 0.0296
Time step 49, Epoch 160: Loss 0.0295
Time step 49, Epoch 170: Loss 0.0285
Time step 49, Epoch 180: Loss 0.0236
Time step 49, Epoch 190: Loss 0.0247
Time step 49, Epoch 200: Loss 0.0214
Time step 49, Epoch 210: Loss 0.0167
Time step 49, Epoch 220: Loss 0.0260
Time step 49, Epoch 230: Loss 0.0197
Time step 49, Epoch 240: Loss 0.0180
Time step 49, Epoch 250: Loss 0.0153
Time step 49, Epoch 260: Loss 0.0180
Time step 49

GCN Embeddings: 100%|████████████████████████████████████████████████████████| 49/49 [06:49<00:00,  8.36s/it]

Time step 49, Epoch 390: Loss 0.0088


In [10]:
all_embeddings = generate_embeddings(
    nodes_df,
    time_step_graphs,
    feature_cols,
    trained_models,
    device=mps_device
)



Generating Embeddings: 100%|█████████████████████████████████████████████████| 49/49 [00:28<00:00,  1.72it/s]


In [11]:
# 5. Merge all features
print("Merging features...")
final_df = nodes_df.merge(gw_features, on='txId', how='left')
final_df = final_df.merge(all_embeddings, on='txId', how='left')
final_df.to_parquet('data/processed/df_model.parquet', partition_cols='SET')

Merging features...
